# Byte-Pair Encoding tokenization

In [76]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [77]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [78]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


### Подготовка начального словаря

In [79]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


In [80]:
# подготовим изначальный словарь - все символы корпуса + EOT
vocab = ["<|endoftext|>"] + alphabet.copy()

In [81]:
# подготовим для каждого слова список символов этого слова
splits = {word: [c for c in word] for word in word_freqs.keys()}
splits['This']

['T', 'h', 'i', 's']

### Рассчет частотности пар

In [82]:
def compute_pair_freqs(splits):
    """
    Для каждого слова считаем частотность токенов пар по всему корпусу
    """
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [83]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


### Поиск самой частой пары

In [84]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


In [85]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

### Объединение пар

In [86]:
def merge_pair(a, b, splits):
    """
    Заменям два данных токена на их объединение 
    """
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                # замена последовательных символов a, b на совокупность ab
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [87]:
splits = merge_pair("n", "e", splits)
print(splits["Ġtrained"])

['Ġ', 't', 'r', 'a', 'i', 'ne', 'd']


In [88]:
vocab_size = 50

# пока наш словарь меньше заданного размера словаря
while len(vocab) < vocab_size:
    # считаем частотность токенов
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    # находим максимально частотную пару
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    # в splits меняем самую частотную пару токенов на последовательность
    splits = merge_pair(*best_pair, splits)
    # добавляем смердженные символы в словарь соотвтесвия пар и их мерджа
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [89]:
print(merges)

{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('Ġ', 'a'): 'Ġa', ('e', 'r'): 'er', ('Ġt', 'o'): 'Ġto', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'e'): 'Ġtoke', ('Ġtoke', 'n'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni'}


In [90]:
print(vocab, len(vocab))

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'Ġt', 'is', 'Ġa', 'er', 'Ġto', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoke', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'Ġab', 'Ġtokeni'] 50


### BPE алгоритм

In [91]:
def tokenize(text):
    # разбиваем текст на слова + спаны
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    # получаем список слов
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    # получаем список символов для каждого слова
    splits = [[l for l in word] for word in pre_tokenized_text]
    # идем по словарю мерджа
    for pair, merge in merges.items():
        # идем по каждому разбитому на символы слову из splits
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                # если символы подходят паре из merges - заменяем эти символы на мердж символов
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            # заменяем разбитое по символам слово на разбитое по токенам слово
            splits[idx] = split

    return sum(splits, [])

In [92]:
tokenize("This is not a tokenization.")

['This',
 'Ġis',
 'Ġ',
 'n',
 'o',
 't',
 'Ġa',
 'Ġtokeni',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 '.']